## Implementation of GRU on the time-series dataset

In [9]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping # Import EarlyStopping

In [2]:
# Suppress warnings for clearer output
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load test features and labels
X_test = np.load('/content/drive/MyDrive/Infosys_Internship/Data Preparation/rolling_window_sequences.npy')  # Replace with actual file path
metadata_test = pd.read_csv("/content/drive/MyDrive/Infosys_Internship/Data Preparation/sequence_metadata_with_RUL.csv")  # Replace with actual file path
y_test = metadata_test["RUL"].values
print("Test feature shape:", X_test.shape)
print("Test target shape:", y_test.shape)

Test feature shape: (17631, 30, 66)
Test target shape: (17631,)


In [6]:
# Split the data into train and validation sets (assuming no separate train set given)
X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=42)

In [7]:
# Define GRU model creation function
def create_gru_model(input_shape, units=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [10]:
# Create the model
model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]), units=64, learning_rate=0.001, dropout_rate=0.2)

# Callbacks for early stopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [11]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: 8032.0117 - mae: 78.5361 - val_loss: 6225.1772 - val_mae: 67.3713
Epoch 2/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - loss: 5901.1201 - mae: 65.2134 - val_loss: 4941.7295 - val_mae: 58.3207
Epoch 3/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: 4688.6484 - mae: 56.5834 - val_loss: 3950.8347 - val_mae: 51.0624
Epoch 4/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - loss: 3798.4495 - mae: 50.2098 - val_loss: 3161.9595 - val_mae: 44.9087
Epoch 5/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 3005.8730 - mae: 43.8083 - val_loss: 2496.5491 - val_mae: 39.4976
Epoch 6/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 2385.1536 - mae: 38.6314 - val_loss: 1984.1937 - val_mae: 34.8798
Epoch 7/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - loss: 1887.5226 - mae: 34.2346 - val_loss: 1503.1688 - val_mae: 30.2716
Epoch 8/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 1416.9694 - mae: 29.4188 - val_loss: 1115.8164 

In [12]:
# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation MSE: {val_loss:.4f}, MAE: {val_mae:.4f}')

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Calculate test MSE
test_mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {test_mse:.4f}')

Validation MSE: 124.3932, MAE: 8.1016
551/551 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step
Test MSE: 107.6167


Task: Build and train a GRU (Gated Recurrent Unit) neural network model for sequence regression on a given dataset. The implementation must include the following features:

Use of callbacks such as EarlyStopping to prevent overfitting and ModelCheckpoint to save the best model.

Hyperparameter tuning for model parameters like number of GRU units, learning rate, and dropout rate.

Cross-validation (e.g., K-Fold) to robustly evaluate model performance across different data splits.

Reporting of evaluation metrics such as mean squared error (MSE) and mean absolute error (MAE).